In [ ]:
# 事前にGCPの認証が必要で、認証方法は環境によって異なる
# colabの場合:
#   セルで下記を実行
#     from google.colab import auth
#     auth.authenticate_user()
#     %env GCLOUD_PROJECT=GCPのプロジェクトID
# PC等のローカル環境の場合:
#   初回のみ、https://cloud.google.com/sdk/docs/install-sdk からgcloud CLIをインストールし、gcloud initを実行
%load_ext google.cloud.bigquery

# 13章 文字列
## 13-1 部分文字列の検出・抽出・置換
### Q: URLから芥川竜之介の作品を判定して抽出
#### Awesome

In [ ]:
%%bigquery
select *
from example.aozora
-- urlに"/000879/"を含む行を抽出
where url like "%/000879/%"

### Q: 出版年月から年と月を位置指定で抽出
#### Awesome

In [ ]:
%%bigquery
select
    text_id,
    title,
    published_at,
    -- published_atの文字列の一部を抽出
    substr(published_at, 1, 4) as year,
    substr(published_at, 6, 2) as month
from example.aozora

### Q: 出版年月のフォーマットを`YYYY年MM月`から`YYYY-MM`に変換（部分文字列の検出、置換）
#### Awesome

In [ ]:
%%bigquery
select
    text_id,
    title,
    case
        -- published_atが"月"で終わる場合は"年"を"-"に置換して"月"を削除、それ以外は"年"を削除
        when published_at like "%月"
            then replace(replace(published_at, "年", "-"), "月", "")
        else replace(published_at, "年", "")
    end as published_at
from example.aozora

### Q: URLから人物IDと作品IDを抽出 (正規表現による抽出)
#### Awesome

In [ ]:
%%bigquery
select
    title,
    url,
    -- urlから正規表現で抽出
    regexp_extract(url, r"cards/0*([1-9][0-9]*)/") as author_id,
    regexp_extract(url, r"card([0-9]+)\.") as text_id
from example.aozora

## 13-2 区切り文字による文字列分割
### Q: 句点で文に分割
#### Awesome

In [ ]:
%%bigquery
select
    text_id,
    title,
    -- "。"でtextを分割
    split(text, "。") as sentence_list
from example.aozora

### Q: 複数の区切り文字で文に分割
#### Awesome

In [ ]:
%%bigquery
select
    text_id,
    title,
    -- "\n"を"。"に置換した上で、"。"でtextを分割
    split(replace(text, "\n", "。"), "。") as sentence_list
from example.aozora

### Q: 分割結果の配列を列に展開
#### Awesome

In [ ]:
%%bigquery
select
    text_id,
    title,
    --（2）sentence_listの0番目と1番目の要素を取得
    sentence_list[offset(0)] as first_sentence,
    sentence_list[offset(1)] as second_sentence
from (
    --（1）"。"でtextを分割
    select
        *,
        split(text, "。") as sentence_list
    from example.aozora
)

### Q: 分割結果の配列を行に展開
#### Awesome

In [ ]:
%%bigquery
select
    text_id,
    title,
    sentence
from (
    --（1）"。"でtextを分割
    select
        *,
        split(text, "。") as sentence_list
    from example.aozora
)
--（2）sentence_listを複数行に展開
cross join unnest(sentence_list) as sentence

## 13-3 文字列データのクレンジング
### Q: ユニコード正規化
#### Awesome

In [ ]:
%%bigquery
select
    text_id,
    title,
    -- textをユニコード正規化
    normalize(text, NFKC) as text
from example.aozora

### Q: 不要な文字列パターンの除去（正規表現による除去）


#### Awesome

In [ ]:
%%bigquery
-- （1）以下の処理結果をテーブルに格納
create or replace table example.aozora_after_cleansing as

with
-- （2）文への分割と行展開
aozora_sentence as (
    select
        text_id,
        title,
        sentence
    from (
        select
            *,
            -- （2）-1 "\n"を"。"に置換したうえで、"。"でtextを分割
            split(replace(text, "\n", "。"), "。") as sentence_list
        from example.aozora
    )
    -- （2）-2 sentence_listを複数行に展開
    cross join unnest(sentence_list) as sentence
),

-- （3）sentenceをユニコード正規化
cleansing_step1 as (
    select
        * except (sentence),
        normalize(sentence, NFKC) as sentence
    from aozora_sentence
),

-- （4）sentenceに含まれる不要なパターンを正規表現で除去
cleansing_step2 as (
    select
        * except (sentence),
        regexp_replace(sentence, r"《.*?》|\[#.*?\]|[\|※「」()]", "") as sentence
    from cleansing_step1
),

-- （5）sentenceの先頭や末尾にあるスペースを除去
cleansing_step3 as (
    select
        * except (sentence),
        trim(sentence) as sentence
    from cleansing_step2
),

-- （6）文字列の長さが1文字以上のsentenceのみ抽出
cleansing_step4 as (
    select *
    from cleansing_step3
    where length(sentence) >= 1
)

select * from cleansing_step4

In [ ]:
%%bigquery
select * from example.aozora_after_cleansing